# Simulat33 (RPG Battle Simulation)

This section provides a high-level overview of the RPG Battle Simulation code to help readers understand the structure and functionality of the software.

## Overview

The RPG Battle Simulation code is designed to facilitate interactive, turn-based battles between characters with unique classes and skills. The simulation is implemented in a Jupyter notebook using `ipywidgets` to create an engaging user interface.

## Code Structure

### 1. CharacterClass Definition

- **Initialization**: Each character class is initialized with attributes like name, description, strengths, weaknesses, inheritance, starting stats, skill progression, and final stats.
- **Skill Initialization**: Skills are initialized with methods to define their effects.
- **Stat Calculation**: Functions to calculate and update stats based on character level and applied buffs.

### 2. Skill Methods

- **create_skill_method**: Defines a method for each skill, detailing its effects (attack, heal, buff).
- **attack, heal, buff**: Functions to execute the respective actions, applying effects to the target or the user.

### 3. Buff and Status Effect Management

- **update_buffs**: Manages the duration and application of buffs.
- **apply_status_effects**: Applies ongoing status effects like poison and healing over time.

### 4. Battle System

- **battle**: Initializes and manages the battle, tracking character stats and handling turns.
- **display_moves**: Displays the available moves for the current character and their descriptions.
- **on_move_select**: Executes the selected move and updates the battle log.

### 5. User Interface

- **ipywidgets**: Used to create interactive elements for move selection and displaying logs.
- **Logs**: Two logs are displayed side-by-side:
  - **Round Log**: Summary of the current round.
  - **Battle Log**: Continuous log of the entire battle.

## Key Functions and Algorithms

### Stat Calculation

Calculates character stats at a given level and updates based on buffs and debuffs.

### Skill Execution

Defines how each skill affects the target, whether it's an attack, heal, or buff.

### Buff and Status Effect Management

Handles the duration and impact of temporary buffs and status effects on characters.

### Turn Management

Tracks whose turn it is, displays available moves, and executes selected moves.

In [2]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

class CharacterClass:
    """
    A class to represent a character with specific attributes and abilities.
    """

    def __init__(self, name, description, strengths, weaknesses, inheritance, starting_stats, progression, final_stats):
        """
        Initialize the character class with provided attributes.
        """
        self.name = name
        self.description = description
        self.strengths = strengths
        self.weaknesses = weaknesses
        self.inheritance = inheritance
        self.starting_stats = starting_stats
        self.progression = progression
        self.final_stats = final_stats
        self.current_stats = starting_stats.copy()
        self.buff_changes = {}  # To track active buffs
        self.buff_timers = {}   # To track duration of active buffs
        self.poison_timers = 0  # To track poison effect duration
        self.heal_over_time_timers = 0  # To track heal over time duration
        self.damage_reduction_timers = 0  # To track damage reduction duration
        self.damage_reduction_amount = 0  # To track damage reduction amount
        self.paralysis_timers = 0  # To track paralysis duration
        self._initialize_skills()  # Initialize the skills for this character

    def _initialize_skills(self):
        """
        Initialize skills by creating methods for each skill in the progression.
        """
        for level, skill in self.progression.items():
            skill["method"] = self.create_skill_method(skill)

    def calculate_stats_at_level(self, level):
        """
        Calculate character stats based on their level.
        """
        for stat in self.starting_stats:
            self.current_stats[stat] = self.starting_stats[stat] + (self.final_stats[stat] - self.starting_stats[stat]) * (level - 1) / 4
        self.calculate_evasion()

    def calculate_evasion(self):
        """
        Calculate the evasion stat based on luck and active buffs.
        """
        luck_influence = min(self.current_stats["luck"] * 0.07 / 100, 0.07)
        evasion_bonus = sum(int(value) for key, value in self.buff_changes.items() if "Evasion" in key)
        total_evasion = min(luck_influence + evasion_bonus / 100, 0.20)
        self.current_stats["evasion"] = total_evasion

    def create_skill_method(self, skill):
        """
        Create a method for each skill to apply its effects.
        """
        def skill_method(target):
            results = []
            for effect in skill["effects"]:
                if effect["type"] == "attack":
                    results.append(self.attack(target, effect))
                elif effect["type"] == "heal":
                    results.append(self.heal(self, effect))
                elif effect["type"] == "buff":
                    results.append(self.buff(effect))
            self.update_buffs()
            status_effects = self.apply_status_effects()
            results.extend(status_effects)
            return results
        return skill_method

    def execute_skill(self, target, skill):
        """
        Execute a skill on a target.
        """
        return skill["method"](target)

    def update_buffs(self):
        """
        Update active buffs and remove expired ones.
        """
        for buff in list(self.buff_timers):
            self.buff_timers[buff] -= 1
            if self.buff_timers[buff] <= 0:
                del self.buff_changes[buff]
                del self.buff_timers[buff]
        self.calculate_evasion()

    def apply_status_effects(self):
        """
        Apply ongoing status effects such as poison and heal over time.
        """
        results = []
        if self.poison_timers > 0:
            poison_damage = self.current_stats["hp"] * 0.05
            self.current_stats["hp"] -= poison_damage
            self.poison_timers -= 1
            results.append(f"{self.name} takes {poison_damage} poison damage. {self.name} HP: {self.current_stats['hp']}")
        
        if self.heal_over_time_timers > 0:
            heal_amount = self.current_stats["s_attack"] * 0.1
            self.current_stats["hp"] = min(self.current_stats["hp"] + heal_amount, self.final_stats["hp"])
            self.heal_over_time_timers -= 1
            results.append(f"{self.name} heals for {heal_amount} over time. {self.name} HP: {self.current_stats['hp']}")
        
        if self.damage_reduction_timers > 0:
            self.damage_reduction_timers -= 1
            results.append(f"{self.name} has {self.damage_reduction_timers} turns of damage reduction remaining.")

        return results

    def attack(self, target, effect):
        """
        Perform an attack on the target, applying damage and status effects.
        """
        if effect["attack_type"] == "spiritual":
            damage = max(0, self.current_stats["s_attack"] - target.current_stats["s_defence"])
        else:
            damage = max(0, self.current_stats["p_attack"] - target.current_stats["p_defence"])

        if random.random() < self.current_stats["luck"] / 100:
            damage *= 1.5  # Critical hit
        if target.damage_reduction_timers > 0:
            damage *= (1 - target.damage_reduction_amount)
        target.current_stats["hp"] -= damage

        if "poison" in effect:
            target.poison_timers = effect["poison"]
        if "paralyze" in effect:
            target.paralysis_timers = effect["paralyze"]
        
        return f"{self.name} uses {effect['name']} on {target.name} for {damage} damage. {target.name} HP: {target.current_stats['hp']}"

    def heal(self, target, effect):
        """
        Heal the character by a certain amount.
        """
        heal_amount = self.current_stats["s_attack"] // 2
        if random.random() < self.current_stats["luck"] / 100:
            heal_amount *= 1.5  # Critical heal
        target.current_stats["hp"] = min(target.current_stats["hp"] + heal_amount, target.final_stats["hp"])
        return f"{self.name} uses {effect['name']} and heals for {heal_amount}. {self.name} HP: {self.current_stats['hp']}"

    def buff(self, effect):
        """
        Apply a buff to the character.
        """
        self.buff_changes[effect["buff_type"]] = effect["buff_value"]
        self.buff_timers[effect["buff_type"]] = effect["duration"]
        return f"{self.name} uses {effect['name']} and gains a buff."

# Example Characters
primal_shifter = CharacterClass(
    name="Primal Shifter",
    description="A druidic trickster who can tap into their primal nature to augment their body with aspects of different creatures, each augmentation bestowing unique abilities.",
    strengths="High evasion, extremely versatile due to diverse augmentations.",
    weaknesses="Low health and physical attack.",
    inheritance={
        "name": "Layered Legacy",
        "description": "Retains some bonuses of the previous aspect upon changing, allowing strategic layering of augmentations."
    },
    starting_stats={
        "hp": 200,
        "p_attack": 50,
        "p_defence": 40,
        "s_attack": 40,
        "s_defence": 40,
        "speed": 70,
        "luck": 60
    },
    progression={
        1: {"name": "Wolf Aspect", "description": "Augments the body with wolf-like features, increasing speed and evasion. Attack: Deals moderate damage.", "effects": [
                {"type": "attack", "attack_type": "physical", "value": 50, "name": "Wolf Aspect Attack"},
                {"type": "buff", "buff_type": "speed", "buff_value": 20, "duration": 3, "name": "Wolf Aspect Speed Buff"},
                {"type": "buff", "buff_type": "evasion", "buff_value": 10, "duration": 3, "name": "Wolf Aspect Evasion Buff"}
            ]
        },
        2: {"name": "Snake Aspect", "description": "Gains snake-like attributes, allowing for poisonous attacks and increased evasion. Attack: Deals moderate damage and poisons the enemy.", "effects": [
                {"type": "attack", "attack_type": "physical", "value": 50, "poison": 3, "name": "Snake Aspect Poison Attack"},
                {"type": "buff", "buff_type": "evasion", "buff_value": 15, "duration": 3, "name": "Snake Aspect Evasion Buff"}
            ]
        },
        3: {"name": "Buzzard Aspect", "description": "Develops buzzard-like traits, allowing for aerial attacks and increased sight. Attack: Deals high damage.", "effects": [
                {"type": "attack", "attack_type": "physical", "value": 80, "name": "Buzzard Aspect Attack"}
            ]
        },
        4: {"name": "Aloe Vera Aspect", "description": "Develops traits of the aloe vera plant, healing over time and providing beneficial auras. Heal: Recovers health over time.", "effects": [
                {"type": "heal", "value": 30, "name": "Aloe Vera Heal Over Time", "duration": 3}
            ]
        },
        5: {"name": "Doppelganger", "description": "Creates a duplicate of the Primal Shifter, effectively doubling evasion and abilities for 2 turns. Buff: Doubles evasion.", "effects": [
                {"type": "buff", "buff_type": "evasion", "buff_value": 2, "duration": 2, "name": "Doppelganger Evasion Buff"}
            ]
        }
    },
    final_stats={
        "hp": 300,
        "p_attack": 100,
        "p_defence": 80,
        "s_attack": 80,
        "s_defence": 80,
        "speed": 140,
        "luck": 120
    }
)

conjuror = CharacterClass(
    name="Conjuror",
    description="A spiritualist who imbues objects with magical energy, communicates with spirits, creates potions, and wields elemental powers.",
    strengths="High spiritual attack, versatile with both offensive and supportive abilities.",
    weaknesses="Low physical power and health.",
    inheritance={
        "name": "Echoing Resonance",
        "description": "Enhances spoken-word magic, causing spells to echo and repeat their effects after a delay, compounding in sustained battles."
    },
    starting_stats={
        "hp": 180,
        "p_attack": 20,
        "p_defence": 30,
        "s_attack": 100,
        "s_defence": 90,
        "speed": 50,
        "luck": 50
    },
    progression={
        1: {"name": "Vampiric Embrace", "description": "Channels the energy of Obeni, siphoning life from the enemy. Attack: Deals moderate damage and heals for the same amount.", "effects": [
                {"type": "attack", "attack_type": "spiritual", "value": 50, "name": "Vampiric Embrace Attack"},
                {"type": "heal", "value": 50, "name": "Vampiric Embrace Heal"}
            ]
        },
        2: {"name": "Illusion Weaver", "description": "Embraces the essence of Lingaki, causing paralysis and a small amount of damage. Attack: Deals small damage and paralyzes the enemy.", "effects": [
                {"type": "attack", "attack_type": "spiritual", "value": 30, "paralyze": 3, "name": "Illusion Weaver Attack"}
            ]
        },
        3: {"name": "Arcane Infusion", "description": "Unleashes the influence of Zuberi, increasing the Conjuror's attack power. Buff: Increases attack power.", "effects": [
                {"type": "buff", "buff_type": "s_attack", "buff_value": 20, "duration": 3, "name": "Arcane Infusion Buff"}
            ]
        },
        4: {"name": "Potion of Simisola", "description": "Crafts a potent potion using Simisola's knowledge, reducing incoming damage. Buff: Reduces incoming damage by 25% for 3 turns.", "effects": [
                {"type": "buff", "buff_type": "damage_reduction", "buff_value": 0.25, "duration": 3, "name": "Potion of Simisola Buff"}
            ]
        },
        5: {"name": "Elemental Awakening", "description": "Calls upon elemental forces to deal high damage to the enemy. Attack: Deals high damage.", "effects": [
                {"type": "attack", "attack_type": "spiritual", "value": 70, "name": "Elemental Awakening Attack"}
            ]
        }
    },
    final_stats={
        "hp": 270,
        "p_attack": 30,
        "p_defence": 50,
        "s_attack": 150,
        "s_defence": 130,
        "speed": 70,
        "luck": 70
    }
)

# Battle System
def battle(class1, level1, class2, level2):
    """
    Initialize the battle with two characters and their levels.
    """
    # Calculate stats for both characters based on their levels
    class1.calculate_stats_at_level(level1)
    class2.calculate_stats_at_level(level2)

    # Initialize battle and round logs
    battle_log = widgets.Textarea(
        value=f"{class1.name} at level {level1} stats: {class1.current_stats}\n{class2.name} at level {level2} stats: {class2.current_stats}\n",
        placeholder='Battle log',
        description='Log:',
        disabled=True,
        layout=widgets.Layout(width='50%', height='300px')
    )

    round_log = widgets.Textarea(
        value="",
        placeholder='Round log',
        description='Round Log:',
        disabled=True,
        layout=widgets.Layout(width='50%', height='300px')
    )

    round_counter = 1  # Initialize round counter

    def display_moves(current_turn):
        """
        Display available moves for the current character.
        """
        clear_output(wait=True)  # Clear previous output
        move_buttons = []
        for move in current_turn["character"].progression.values():
            button = widgets.Button(description=move['name'])
            button.on_click(lambda b, m=move: on_move_select(current_turn, m))
            move_buttons.append(button)

        # Display the current character and the opponent
        current_character_label = widgets.HTML(value=get_character_display(current_turn["character"]), layout=widgets.Layout(border='2px solid red'))
        other_character = class2 if current_turn["character"] == class1 else class1
        other_character_label = widgets.HTML(value=get_character_display(other_character))

        move_buttons_box = widgets.VBox(move_buttons)
        move_description = widgets.HTML(value="<b>Move Descriptions:</b><br>" + "<br>".join([f"{move['name']}: {move['description']}" for move in current_turn["character"].progression.values()]))

        # Display the battle log and round log side by side
        display(widgets.VBox([widgets.HBox([widgets.VBox([current_character_label]), widgets.VBox([other_character_label])]), widgets.HBox([move_buttons_box, move_description]), widgets.HBox([battle_log, round_log])]))

    def get_character_display(character):
        """
        Generate the HTML display for a character including stats, buffs, and status effects.
        """
        buffs = ''.join([f'<div>{k}: {v}</div>' for k, v in character.buff_changes.items()])
        status_effects = ""
        if character.poison_timers > 0:
            status_effects += f'<div>Poison: {character.poison_timers} turns remaining</div>'
        if character.paralysis_timers > 0:
            status_effects += f'<div>Paralysis: {character.paralysis_timers} turns remaining</div>'
        return f"<b>{character.name}</b><br>HP: <b>{character.current_stats['hp']}</b><br>{buffs}{status_effects}"

    def on_move_select(current_turn, move):
        """
        Handle the move selection and apply its effects.
        """
        nonlocal class1, class2, round_counter

        opponent = class2 if current_turn["character"] == class1 else class1

        if current_turn["character"].paralysis_timers > 0 and random.random() < 0.2:
            battle_log.value = f"{current_turn['character'].name} is paralyzed and cannot move!\n" + battle_log.value
        else:
            results = current_turn["character"].execute_skill(opponent, move)
            for result in results:
                battle_log.value = result + "\n" + battle_log.value
        
        status_effects = current_turn["character"].apply_status_effects()
        for effect in status_effects:
            battle_log.value = effect + "\n" + battle_log.value

        round_summary = (
            f"Round {round_counter} Summary:\n"
            f"{class1.name} - HP: {class1.current_stats['hp']}, Buffs: {class1.buff_changes}, "
            f"Poison: {class1.poison_timers}, Paralysis: {class1.paralysis_timers}\n"
            f"{class2.name} - HP: {class2.current_stats['hp']}, Buffs: {class2.buff_changes}, "
            f"Poison: {class2.poison_timers}, Paralysis: {class2.paralysis_timers}\n"
        )
        round_log.value = round_summary + "\n" + round_log.value

        if class1.current_stats["hp"] <= 0:
            battle_log.value = f"{class1.name} is defeated!\n" + battle_log.value
            display(battle_log)
            return
        if class2.current_stats["hp"] <= 0:
            battle_log.value = f"{class2.name} is defeated!\n" + battle_log.value
            display(battle_log)
            return

        round_counter += 1  # Increment the round counter
        next_turn = {"character": class2} if current_turn["character"] == class1 else {"character": class1}
        display_moves(next_turn)

    initial_turn = {"character": class1}
    display_moves(initial_turn)

# Start the interactive battle
battle(primal_shifter, 5, conjuror, 5)
